In [25]:
#%pip install llama-index-llms-google-genai llama-index python-dotenv

In [26]:
from dotenv import load_dotenv
import os
from llama_index.llms.gemini import Gemini
from llama_index.core.agent.workflow import AgentWorkflow

load_dotenv()
GEMINI_API_KEY = os.environ["GEMINI_API_KEY"]

In [28]:
from llama_index.llms.google_genai import GoogleGenAI

llm = GoogleGenAI(
    model="gemini-1.5-flash",
    api_key=GEMINI_API_KEY,
)

## Time Tool

In [29]:
from llama_index.core.tools import FunctionTool
from llama_index.core.llms import ChatMessage
from datetime import datetime
def get_current_time(timezone: str) -> dict:
    """Get the current time"""
    return {
        "time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "timezone": timezone,
    }

time_tool = FunctionTool.from_defaults(fn=get_current_time)

In [30]:
#%pip install --upgrade llama-index


In [ ]:
from llama_index.core.agent.workflow import AgentWorkflow
# Use agent workflow directly
workflow = AgentWorkflow.from_tools_or_functions(
    [time_tool],
    llm=llm,
    system_prompt=("You are a helpful assistant. "
    "You can call functions to get the current time in any timezone. "
    "Use the 'get_current_time' function when the user asks for the current time.")
)

response = await workflow.run(user_msg="What is the current time in India?")
print(response)


The current time in India is 2025-06-18 12:17:56.



## Basic Calculation Agent

In [40]:
# Defining basic tools
def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b

def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b

def subtract(a: float, b: float) -> float:
    """Subtract two numbers and returns the difference"""
    return a - b


In [32]:
from llama_index.core.agent.workflow import AgentWorkflow

# Use agent workflow directly
workflow2 = AgentWorkflow.from_tools_or_functions(
    [add, subtract, multiply],
    llm=llm,
    system_prompt=("You are a helpful assistant. "
    "You can call functions to do arithmetic calculations. "
    "Use the 'add', 'subtract', and 'multiply' functions when the user asks for arithmetic operations.")
)

response = await workflow2.run(user_msg="What is 5+ 3, 10 - 4, and 2 * 6?")
print(response)


5 + 3 = 8, 10 - 4 = 6, and 2 * 6 = 12.



## Tavily Research API

In [37]:
#%pip install llama-index-tools-tavily-research

In [34]:
from llama_index.tools.tavily_research import TavilyToolSpec
import os

In [35]:
TAVILY_API_KEY = os.environ["TAVILY_API_KEY"]
tavily_tool = TavilyToolSpec(api_key=TAVILY_API_KEY)

In [39]:
workflow3 = AgentWorkflow.from_tools_or_functions(
    tavily_tool.to_tool_list() + [time_tool],
    llm=llm,
    system_prompt=("You're a helpful assistant that can search the web for information."
                   "You are a helpful assistant. "
                    "You can call functions to get the current time in any timezone. "
                    "Use the 'get_current_time' function when the user asks for the current time."
                    "You can also use the Tavily tool to search for information online.")
)

response = await workflow3.run(user_msg="What is the capital of France? What is the current time there?")
print(response)

The capital of France is Paris. The current time in Paris is 2025-06-18 12:19:20.

